In [ ]:
"""

# Project Title
End-to-End Customer Comments Analysis Workflow Using LCEL Chain Routing with  LangChain and OpenAI

# Project Objective:
Key objectives are:
•	Divide user messages into certain categories using an LLM.
•	Send each message to the matching branch using LCEL’s route.
•	Develop customer findings: highlights, sentiment, interpret, and summarize.
•	Make a clean system for Agentic AI Workflows.

# Essentials:
•	Classification on customers feedback from LLM.
•	Branching from LCEL RunnableBranch task.
•	Using prompt engineering to get intended chain.
•	Generate end to end chain.
•	Testing different cases for customer feedback.
•	unsupported categories solved with a default function (no_match).

# Project file:
LCEL_Chain_Routing _Customer_Feedback_Classification.ipynb

"""

In [ ]:
# To run this project, I Installed all required dependencies for OpenAI and LangChain into Anaconda Navigator. 

In [1]:
import os

In [3]:
# Load openai api key
from dotenv import load_dotenv

load_dotenv()

openapi_key = os.getenv('OPENAI_API_KEY')

In [5]:
from langchain.prompts import ChatPromptTemplate

In [7]:
from langchain_openai import ChatOpenAI

chatopenai_gpt = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [10]:
# prompt chain 

llm_classification_prompt = ChatPromptTemplate.from_template(
    '''Please read the user guidelines carefully below for customer opinion analysis
    and then identify it as **one** of the following categories:  
    -  highlights
    -  sentiment
    -  interpret
    -  summarize
    
    Please provide a one word response

    Directions:
    {directions}
    ''')

from langchain_core.output_parsers import StrOutputParser

llm_classification_prompt_chain = (llm_classification_prompt | chatopenai_gpt | StrOutputParser())

# Check:
llm_classification_prompt_chain.invoke({'directions': 'Please provide sentiment this customer feedback.'})



'sentiment'

In [12]:
# Highlights prompt
llm_highlights_prompt = ChatPromptTemplate.from_template(
    '''Please read the customer comments given below and identify the main highlights. 
    Please provide a list which is separated by semicolons.
    
    Customer Comments:
    {comments}
    ''')

llm_highlights_prompt_chain = (llm_highlights_prompt | chatopenai_gpt | StrOutputParser())


In [14]:
# sentiment prompt

llm_sentiment_prompt = ChatPromptTemplate.from_template(
    '''Please read the customer comments given below and then identify the sentiment as one of these:  happy , unfavorable,  or emotionless.
    Please restrict your answer to a single word. 
    
    Customer Comments:
    {comments}
    ''')

llm_sentiment_prompt_chain = (llm_sentiment_prompt | chatopenai_gpt | StrOutputParser())


In [16]:
# Interpret prompt

llm_interpret_prompt = ChatPromptTemplate.from_template(
    '''Please read the customer comments below and give your interpretation of the customer’s feedback. 
    Please provide your answer within 2 sentences. 
    
    Customer Comments:
    {comments}
    ''')

llm_interpret_prompt_chain = (llm_interpret_prompt | chatopenai_gpt | StrOutputParser())


In [18]:
# summary prompt

llm_summary_prompt = ChatPromptTemplate.from_template(
    '''Please read the customer comments as given below and produce a concise summary of up to 3 sentences.

    Customer Comments:
    {comments}
    ''')

llm_summary_prompt_chain = (llm_summary_prompt | chatopenai_gpt | StrOutputParser())

# Check:
llm_summary_prompt_chain.invoke({'comments': 'Please provide summary for this customer feedback.'})


"It seems that there are no specific customer comments provided for me to summarize. Please share the comments, and I'll be happy to create a concise summary for you!"

In [21]:
# The request is not related with any identified category. 
def no_match(user_input):
    return 'The request is not related with any identified category.'

In [23]:
from langchain_core.runnables import RunnableBranch


# LLM branch routing
# .lower() used for ignoring capitalization. 

router_branch = RunnableBranch((lambda a: 'highlights' in a['task'].lower(), llm_highlights_prompt_chain),
    (lambda a: 'sentiment' in a['task'].lower(), llm_sentiment_prompt_chain),
    (lambda a: 'interpret' in a['task'].lower(), llm_interpret_prompt_chain),
    (lambda a: 'summarize' in a['task'].lower() or 'summary' in a['task'].lower(), llm_summary_prompt_chain),
    no_match,)

In [25]:
end_to_end_chain = ({'task': llm_classification_prompt_chain,
        'instruction': lambda task_input: task_input.get('instruction'),
        'comments': lambda task_input: task_input.get('comments')} | router_branch)


In [27]:
customer_comment = '''I ordered a chair through online. When we use it, It reduces back pain. The chair has a fashionable design as well as looks 
carefully crafted. However, the seat is harder than expected. It was delivered early. It was unexpected. On the other hand, minor damage was seen 
to one armrest. I communicated to customer service through email. I got late reply but the replacement part was finally sent. After putting the 
replacement part together, the chair looks robust, but I am not sure whether it is comfortable or not. I honestly appreciate the company’s efforts 
to solve this problem. On the other hand, with this price, I had expected frictionless experience.'''


In [29]:
Customer_comment_highlights = end_to_end_chain.invoke({
    'directions': 'Please provide the main highlights of the customer comment which is provided below',
    'comments': customer_comment})

print(Customer_comment_highlights)

Main highlights: Reduces back pain; Fashionable design; Carefully crafted; Seat harder than expected; Delivered early; Minor damage to one armrest; Late reply from customer service; Replacement part sent; Chair looks robust after assembly; Uncertainty about comfort; Appreciation for company's problem-solving efforts; Expected a frictionless experience for the price.


In [31]:
Customer_comment_sentiment = end_to_end_chain.invoke({
    'directions': 'Please provide the sentiment of the customer comment which is provided below',
    'comments': customer_comment})

print(Customer_comment_sentiment)


unfavorable


In [33]:
Customer_comment_interpret = end_to_end_chain.invoke({
    'directions': 'Please interpret the customer comment which is provided below',
    'comments': customer_comment})

print(Customer_comment_interpret)


The customer appreciates the chair's design and its effectiveness in reducing back pain, but expresses disappointment with the seat's hardness and minor damage upon delivery. While they acknowledge the company's efforts in addressing the issue, they expected a smoother experience given the price point.


In [ ]:
# Get summary from 3 different directions.

In [35]:
Customer_comment_summarize = end_to_end_chain.invoke({
    'directions': 'Summarize the customer comment which is provided below up to 3 sentences',
    'comments': customer_comment})
print(Customer_comment_summarize)


The customer appreciates the chair's stylish design and its effectiveness in reducing back pain, but finds the seat harder than anticipated. Although the chair was delivered early, there was minor damage to an armrest, and while customer service was slow to respond, they eventually sent a replacement part. Overall, the customer values the company's efforts to resolve the issue but expected a smoother experience given the price.


In [37]:
Customer_comment_summarize = end_to_end_chain.invoke({
    'directions': 'Please summarize the customer comment which is provided below',
    'comments': customer_comment})
print(Customer_comment_summarize)


The customer appreciates the chair's stylish design and its effectiveness in reducing back pain, but finds the seat harder than anticipated. Although the chair was delivered early and customer service eventually provided a replacement part for minor damage, the customer experienced delays in communication and expected a smoother overall experience given the price. Overall, while the chair appears robust after the repair, the customer remains uncertain about its comfort.


In [39]:
Customer_comment_summarize = end_to_end_chain.invoke({
    'directions': 'summarize',
    'comments': customer_comment
})

print(Customer_comment_summarize)

The customer appreciates the chair's stylish design and its effectiveness in reducing back pain, but finds the seat harder than anticipated. Although the chair was delivered early and the company provided a replacement part for minor damage, the customer experienced delays in communication and expected a smoother overall experience given the price. Overall, while the chair appears robust after the repair, the customer remains uncertain about its comfort.
